In [ ]:
# # Import necessary libraries
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# from sklearn.metrics import classification_report
# from catboost import CatBoostClassifier
# import numpy as np
# from scipy.sparse import hstack

# # Load the datasets
# train = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
# test = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")

# # Encode the target columns
# train['target'] = np.select(
#     [train['winner_model_a'] == 1, train['winner_model_b'] == 1, train['winner_tie'] == 1],
#     [0, 1, 2],
#     default=-1
# )

# # Remove unnecessary columns
# train.drop(columns=['winner_model_a', 'winner_model_b', 'winner_tie', 'id'], inplace=True)

# # Prepare features and target variable
# X = train[['model_a','model_b','prompt', 'response_a', 'response_b']]
# y = train['target']

# # Combine the text columns into a single feature for vectorization
# X_combined =X['model_a'] +X['model_b']+ X['prompt'] + " " + X['response_a'] + " " + X['response_b']

# # Split the data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X_combined, y, test_size=0.02, random_state=42, stratify=y)

# # Count Vectorization without stop words
# count_vectorizer = CountVectorizer(max_features=50000, stop_words=None)
# X_train_counts = count_vectorizer.fit_transform(X_train)
# X_val_counts = count_vectorizer.transform(X_val)

# # TF-IDF Vectorization without stop words
# tfidf_vectorizer = TfidfVectorizer(max_features=50000, stop_words=None)
# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
# X_val_tfidf = tfidf_vectorizer.transform(X_val)

# # Combine Count and TF-IDF features
# X_train_combined = hstack([X_train_counts, X_train_tfidf])
# X_val_combined = hstack([X_val_counts, X_val_tfidf])

# # Initialize CatBoost Classifier with appropriate parameters
# model = CatBoostClassifier(iterations=5000, depth=6, learning_rate=0.01, verbose=100, loss_function='MultiClass')

# # Train the model
# model.fit(X_train_combined ,  y_train)

# # Validate the model
# val_predictions = model.predict(X_val_combined)
# print("Classification Report for CatBoost:")
# print(classification_report(y_val, val_predictions))

# # Prepare the test data by combining the text columns
# X_test_combined =test['model_a']+ test['model_b'] + test['prompt'] + " " + test['response_a'] + " " + test['response_b']

# # Transform test data with both Count and TF-IDF vectorizers
# X_test_counts = count_vectorizer.transform(X_test_combined)
# X_test_tfidf = tfidf_vectorizer.transform(X_test_combined)

# # Combine test features
# X_test_combined = hstack([X_test_counts, X_test_tfidf])

# # Predict probabilities for each class
# test_probabilities = model.predict_proba(X_test_combined)

# # Create submission DataFrame
# submission = pd.DataFrame(test_probabilities, columns=['winner_model_a', 'winner_model_b', 'winner_tie'])
# submission['id'] = test['id']

# # Reorder columns for submission
# submission = submission[['id', 'winner_model_a', 'winner_model_b', 'winner_tie']]

# # Save submission to CSV
# submission_file = '/kaggle/working/submission.csv'
# submission.to_csv(submission_file, index=False)
# print(f"Submission saved to {submission_file}")


In [ ]:
# # Import necessary libraries
# import pandas as pd
# import numpy as np
# import pickle
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report
# from sklearn.ensemble import HistGradientBoostingClassifier
# import os

# # Define paths to embedding files
# TRAIN_EMBEDDINGS_DIR = "/kaggle/input/training-llm-prompt-classifciation-bert-embedding/"
# TEST_EMBEDDINGS_DIR = "/kaggle/working/"

# # Paths for training embeddings
# TRAIN_PROMPT_EMBEDDING_PATH = os.path.join(TRAIN_EMBEDDINGS_DIR, "prompt_bert.pkl")
# TRAIN_RESPONSE_A_EMBEDDING_PATH = os.path.join(TRAIN_EMBEDDINGS_DIR, "response_a_bert.pkl")
# TRAIN_RESPONSE_B_EMBEDDING_PATH = os.path.join(TRAIN_EMBEDDINGS_DIR, "response_b_bert.pkl")

# # Paths for test embeddings
# TEST_PROMPT_EMBEDDING_PATH = os.path.join(TEST_EMBEDDINGS_DIR, "prompt_bert.pkl")
# TEST_RESPONSE_A_EMBEDDING_PATH = os.path.join(TEST_EMBEDDINGS_DIR, "response_a_bert.pkl")
# TEST_RESPONSE_B_EMBEDDING_PATH = os.path.join(TEST_EMBEDDINGS_DIR, "response_b_bert.pkl")

# # Load training data
# train_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
# test_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")

# # Encode the target columns
# train_df['target'] = np.select(
#     [
#         train_df['winner_model_a'] == 1,
#         train_df['winner_model_b'] == 1,
#         train_df['winner_tie'] == 1
#     ],
#     [0, 1, 2],
#     default=-1  # Assign -1 for any invalid cases
# )

# # Verify that there are no invalid targets
# if (train_df['target'] == -1).any():
#     raise ValueError("There are invalid target labels in the training data.")

# # Define target variable
# y = train_df['target']

# # Load BERT embeddings for training data
# with open(TRAIN_PROMPT_EMBEDDING_PATH, 'rb') as f:
#     prompt_embeddings_train = pickle.load(f)

# with open(TRAIN_RESPONSE_A_EMBEDDING_PATH, 'rb') as f:
#     response_a_embeddings_train = pickle.load(f)

# with open(TRAIN_RESPONSE_B_EMBEDDING_PATH, 'rb') as f:
#     response_b_embeddings_train = pickle.load(f)

# # Verify that all embeddings have the same number of samples
# num_train_samples = train_df.shape[0]
# assert prompt_embeddings_train.shape[0] == num_train_samples, "Mismatch in number of samples for prompt embeddings."
# assert response_a_embeddings_train.shape[0] == num_train_samples, "Mismatch in number of samples for response A embeddings."
# assert response_b_embeddings_train.shape[0] == num_train_samples, "Mismatch in number of samples for response B embeddings."

# # Concatenate all embeddings to form the feature matrix
# X_train_full = np.hstack([
#     prompt_embeddings_train,
#     response_a_embeddings_train,
#     response_b_embeddings_train
# ])

# # Load BERT embeddings for test data
# with open(TEST_PROMPT_EMBEDDING_PATH, 'rb') as f:
#     prompt_embeddings_test = pickle.load(f)

# with open(TEST_RESPONSE_A_EMBEDDING_PATH, 'rb') as f:
#     response_a_embeddings_test = pickle.load(f)

# with open(TEST_RESPONSE_B_EMBEDDING_PATH, 'rb') as f:
#     response_b_embeddings_test = pickle.load(f)

# # Verify that all test embeddings have the same number of samples
# num_test_samples = test_df.shape[0]
# assert prompt_embeddings_test.shape[0] == num_test_samples, "Mismatch in number of samples for test prompt embeddings."
# assert response_a_embeddings_test.shape[0] == num_test_samples, "Mismatch in number of samples for test response A embeddings."
# assert response_b_embeddings_test.shape[0] == num_test_samples, "Mismatch in number of samples for test response B embeddings."

# # Concatenate all test embeddings to form the test feature matrix
# X_test = np.hstack([
#     prompt_embeddings_test,
#     response_a_embeddings_test,
#     response_b_embeddings_test
# ])

# # Split the training data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(
#     X_train_full,
#     y,
#     test_size=0.02,
#     random_state=42,
#     stratify=y
# )

# # Initialize HistGradientBoostingClassifier
# hgb_model = HistGradientBoostingClassifier(
#     max_iter=100,
#     max_depth=10,
#     random_state=42,
#     class_weight="balanced"  # Adjust for potential class imbalance
# )

# # Train the HistGradientBoosting model
# print("Training HistGradientBoosting model...")
# hgb_model.fit(X_train, y_train)
# print("HistGradientBoosting training completed.")

# # Validate the HistGradientBoosting model
# print("Evaluating HistGradientBoosting model on validation set...")
# hgb_val_predictions = hgb_model.predict(X_val)
# print("Classification Report for HistGradientBoosting:")
# print(classification_report(y_val, hgb_val_predictions))

# # Make predictions on the test set using HistGradientBoosting
# print("Making predictions on test data with HistGradientBoosting...")
# hgb_test_predictions = hgb_model.predict_proba(X_test)

# # Ensure that hgb_test_predictions has the correct number of classes
# num_classes = len(hgb_model.classes_)
# if hgb_test_predictions.shape[1] != num_classes:
#     raise ValueError(f"Expected {num_classes} classes, but got {hgb_test_predictions.shape[1]}.")

# # Create HistGradientBoosting submission DataFrame
# hgb_submission = pd.DataFrame(hgb_test_predictions, columns=['winner_model_a', 'winner_model_b', 'winner_tie'])
# hgb_submission['id'] = test_df['id']

# # Reorder columns for submission
# hgb_submission = hgb_submission[['id', 'winner_model_a', 'winner_model_b', 'winner_tie']]

# # Save the HistGradientBoosting submission file
# hgb_submission_file = '/kaggle/working/submission.csv'
# hgb_submission.to_csv(hgb_submission_file, index=False)
# print(f"HistGradientBoosting submission saved to {hgb_submission_file}")

# # Optionally, display the first few rows of the HistGradientBoosting submission file
# print("First few rows of the HistGradientBoosting submission file:")
# print(hgb_submission.head())


In [ ]:
# # Import necessary libraries
# import pandas as pd
# import numpy as np
# import pickle
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report
# from sklearn.ensemble import GradientBoostingClassifier
# import os

# # Define paths to embedding files
# TRAIN_EMBEDDINGS_DIR = "/kaggle/input/training-llm-prompt-classifciation-bert-embedding/"
# TEST_EMBEDDINGS_DIR = "/kaggle/working/"

# # Paths for training embeddings
# TRAIN_PROMPT_EMBEDDING_PATH = os.path.join(TRAIN_EMBEDDINGS_DIR, "prompt_bert.pkl")
# TRAIN_RESPONSE_A_EMBEDDING_PATH = os.path.join(TRAIN_EMBEDDINGS_DIR, "response_a_bert.pkl")
# TRAIN_RESPONSE_B_EMBEDDING_PATH = os.path.join(TRAIN_EMBEDDINGS_DIR, "response_b_bert.pkl")

# # Paths for test embeddings
# TEST_PROMPT_EMBEDDING_PATH = os.path.join(TEST_EMBEDDINGS_DIR, "prompt_bert.pkl")
# TEST_RESPONSE_A_EMBEDDING_PATH = os.path.join(TEST_EMBEDDINGS_DIR, "response_a_bert.pkl")
# TEST_RESPONSE_B_EMBEDDING_PATH = os.path.join(TEST_EMBEDDINGS_DIR, "response_b_bert.pkl")

# # Load training data
# train_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
# test_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")

# # Encode the target columns
# train_df['target'] = np.select(
#     [
#         train_df['winner_model_a'] == 1,
#         train_df['winner_model_b'] == 1,
#         train_df['winner_tie'] == 1
#     ],
#     [0, 1, 2],
#     default=-1  # Assign -1 for any invalid cases
# )

# # Verify that there are no invalid targets
# if (train_df['target'] == -1).any():
#     raise ValueError("There are invalid target labels in the training data.")

# # Define target variable
# y = train_df['target']

# # Load BERT embeddings for training data
# with open(TRAIN_PROMPT_EMBEDDING_PATH, 'rb') as f:
#     prompt_embeddings_train = pickle.load(f)

# with open(TRAIN_RESPONSE_A_EMBEDDING_PATH, 'rb') as f:
#     response_a_embeddings_train = pickle.load(f)

# with open(TRAIN_RESPONSE_B_EMBEDDING_PATH, 'rb') as f:
#     response_b_embeddings_train = pickle.load(f)

# # Verify that all embeddings have the same number of samples
# num_train_samples = train_df.shape[0]
# assert prompt_embeddings_train.shape[0] == num_train_samples, "Mismatch in number of samples for prompt embeddings."
# assert response_a_embeddings_train.shape[0] == num_train_samples, "Mismatch in number of samples for response A embeddings."
# assert response_b_embeddings_train.shape[0] == num_train_samples, "Mismatch in number of samples for response B embeddings."

# # Concatenate all embeddings to form the feature matrix
# X_train_full = np.hstack([
#     prompt_embeddings_train,
#     response_a_embeddings_train,
#     response_b_embeddings_train
# ])

# # Load BERT embeddings for test data
# with open(TEST_PROMPT_EMBEDDING_PATH, 'rb') as f:
#     prompt_embeddings_test = pickle.load(f)

# with open(TEST_RESPONSE_A_EMBEDDING_PATH, 'rb') as f:
#     response_a_embeddings_test = pickle.load(f)

# with open(TEST_RESPONSE_B_EMBEDDING_PATH, 'rb') as f:
#     response_b_embeddings_test = pickle.load(f)

# # Verify that all test embeddings have the same number of samples
# num_test_samples = test_df.shape[0]
# assert prompt_embeddings_test.shape[0] == num_test_samples, "Mismatch in number of samples for test prompt embeddings."
# assert response_a_embeddings_test.shape[0] == num_test_samples, "Mismatch in number of samples for test response A embeddings."
# assert response_b_embeddings_test.shape[0] == num_test_samples, "Mismatch in number of samples for test response B embeddings."

# # Concatenate all test embeddings to form the test feature matrix
# X_test = np.hstack([
#     prompt_embeddings_test,
#     response_a_embeddings_test,
#     response_b_embeddings_test
# ])

# # Split the training data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(
#     X_train_full,
#     y,
#     test_size=0.02,
#     random_state=42,
#     stratify=y
# )

# # Initialize GradientBoostingClassifier
# gb_model = GradientBoostingClassifier(
#     n_estimators=100,
#     max_depth=10,
#     random_state=42
# )

# # Train the GradientBoosting model
# print("Training GradientBoosting model...")
# gb_model.fit(X_train, y_train)
# print("GradientBoosting training completed.")

# # Validate the GradientBoosting model
# print("Evaluating GradientBoosting model on validation set...")
# gb_val_predictions = gb_model.predict(X_val)
# print("Classification Report for GradientBoosting:")
# print(classification_report(y_val, gb_val_predictions))

# # Make predictions on the test set using GradientBoosting
# print("Making predictions on test data with GradientBoosting...")
# gb_test_predictions = gb_model.predict_proba(X_test)

# # Ensure that gb_test_predictions has the correct number of classes
# num_classes = len(gb_model.classes_)
# if gb_test_predictions.shape[1] != num_classes:
#     raise ValueError(f"Expected {num_classes} classes, but got {gb_test_predictions.shape[1]}.")

# # Create GradientBoosting submission DataFrame
# gb_submission = pd.DataFrame(gb_test_predictions, columns=['winner_model_a', 'winner_model_b', 'winner_tie'])
# gb_submission['id'] = test_df['id']

# # Reorder columns for submission
# gb_submission = gb_submission[['id', 'winner_model_a', 'winner_model_b', 'winner_tie']]

# # Save the GradientBoosting submission file
# gb_submission_file = '/kaggle/working/submission.csv'
# gb_submission.to_csv(gb_submission_file, index=False)
# print(f"GradientBoosting submission saved to {gb_submission_file}")

# # Optionally, display the first few rows of the GradientBoosting submission file
# print("First few rows of the GradientBoosting submission file:")
# print(gb_submission.head())


In [ ]:
# print("===========Getting Embedding(Vectors Tensor) of ============")

# import pandas as pd
# import numpy as np
# import re
# import nltk
# import torch
# import pickle
# from collections import Counter
# from nltk.corpus import stopwords
# from transformers import BertTokenizer, BertModel, XLNetTokenizer, XLNetModel, GPT2Tokenizer, GPT2Model, T5Tokenizer, T5Model
# import spacy

# # Ensure NLTK's stopwords are downloaded
# nltk.download('stopwords')

# # Check if GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # Load the data
# train = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv', nrows=5000)
# test = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv', nrows=5000)

# # Load models and transfer them to GPU if available
# print("Loading models...")

# # BERT
# bert_tokenizer = BertTokenizer.from_pretrained('/kaggle/input/vector-model-embedder/models/bert_tokenizer')
# bert_model = BertModel.from_pretrained('/kaggle/input/vector-model-embedder/models/bert_model').to(device)


# # XLNet
# xlnet_tokenizer = XLNetTokenizer.from_pretrained("/kaggle/input/vector-model-embedder/models/xlnet_tokenizer")
# xlnet_model = XLNetModel.from_pretrained("/kaggle/input/vector-model-embedder/models/xlnet_model").to(device)


# # GPT-2
# gpt2_tokenizer = GPT2Tokenizer.from_pretrained('/kaggle/input/vector-model-embedder/models/gpt2_tokenizer')
# gpt2_model = GPT2Model.from_pretrained('/kaggle/input/vector-model-embedder/models/gpt2_model').to(device)

# # T5
# t5_tokenizer = T5Tokenizer.from_pretrained('/kaggle/input/vector-model-embedder/models/t5_tokenizer')
# t5_model = T5Model.from_pretrained('/kaggle/input/vector-model-embedder/models/t5_model').to(device)


# # SpaCy
# nlp_local = spacy.load("/kaggle/input/vector-model-embedder/models/spacy_model")


# # Text preprocessing function
# def preprocess_text(text):
#     text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
#     words = text.lower().split()  # Lowercase and split the text into words
#     stop_words = set(stopwords.words('english'))  # Set of stopwords
#     words = [word for word in words if word not in stop_words]  # Remove stopwords
#     unique_words = list(Counter(words).keys())  # Get unique words
#     return ' '.join(unique_words)  # Join them back into a string

# # Apply preprocessing to all columns that will be used for embedding extraction
# train = train.applymap(preprocess_text)
# test = test.applymap(preprocess_text)

# # Function to extract embeddings from a model
# def extract_embeddings(text, model, tokenizer, max_length=512):
#     inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()  # Mean pooling over token embeddings
#     return embedding

# # Function to extract embeddings using SpaCy
# def text_to_vector(text):
#     doc = nlp_local(text)
#     return doc.vector

# # Function to save embeddings to pickle files
# def save_embeddings(data, filename):
#     with open(filename, 'wb') as f:
#         pickle.dump(data, f)
#     print(f"Saved embeddings to {filename}")

# # Function to generate and save embeddings for a list of columns
# def generate_and_save_embeddings(df, column_names):
#     for column_name in column_names:
#         print(f"Processing column: {column_name}")
#         column_data = df[column_name].dropna()  # Remove any NaN values if present
        
#         # BERT embeddings
#         bert_embeddings = np.array([extract_embeddings(text, bert_model, bert_tokenizer) for text in column_data])
#         save_embeddings(bert_embeddings, f'/kaggle/working/{column_name}_bert.pkl')

#         # XLNet embeddings
#         xlnet_embeddings = np.array([extract_embeddings(text, xlnet_model, xlnet_tokenizer) for text in column_data])
#         save_embeddings(xlnet_embeddings, f'/kaggle/working/{column_name}_xlnet.pkl')

#         # GPT-2 embeddings
#         gpt2_embeddings = np.array([extract_embeddings(text, gpt2_model, gpt2_tokenizer) for text in column_data])
#         save_embeddings(gpt2_embeddings, f'/kaggle/working/{column_name}_gpt2.pkl')

#         # T5 embeddings
#         t5_embeddings = np.array([extract_embeddings(text, t5_model, t5_tokenizer) for text in column_data])
#         save_embeddings(t5_embeddings, f'/kaggle/working/{column_name}_t5.pkl')

#         # SpaCy embeddings
#         spacy_embeddings = np.array([text_to_vector(text) for text in column_data])
#         save_embeddings(spacy_embeddings, f'/kaggle/working/{column_name}_spacy.pkl')

# # List of columns for which embeddings need to be generated
# columns_to_embed = ['prompt', 'response_a', 'response_b']  # Replace with actual column names

# # Generate and save embeddings for the specified columns in both train and test data
# generate_and_save_embeddings(train, columns_to_embed)
# generate_and_save_embeddings(test, columns_to_embed)

# print("All embeddings generated and saved successfully.")



In [ ]:
print("""================== Loading Train Features and Testing Features =============""")

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import pickle

"""=================== Getting embedding for test file ==============="""
import pandas as pd
import numpy as np
import re
import nltk
import torch
import pickle
from collections import Counter
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# Ensure NLTK's stopwords are downloaded
nltk.download('stopwords')

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

train = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')


# Load BERT model and tokenizer, and transfer the model to GPU if available
print("Loading BERT model...")
bert_tokenizer = BertTokenizer.from_pretrained('/kaggle/input/vector-model-embedder/models/bert_tokenizer')
bert_model = BertModel.from_pretrained('/kaggle/input/vector-model-embedder/models/bert_model').to(device)


# Text preprocessing function
def preprocess_text(text):
    if isinstance(text, str):  # Ensure text is a string before processing
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        words = text.lower().split()  # Lowercase and split the text into words
        stop_words = set(stopwords.words('english'))  # Set of stopwords
        words = [word for word in words if word not in stop_words]  # Remove stopwords
        unique_words = list(Counter(words).keys())  # Get unique words
        return ' '.join(unique_words)  # Join them back into a string
    return ''  # Return empty string if text is not a valid string


# Apply preprocessing only to specific columns
columns_to_preprocess = ['prompt', 'response_a', 'response_b']  # Replace with actual column names
for col in columns_to_preprocess:
    train[col] = train[col].apply(preprocess_text)
    # test[col] = test[col].apply(preprocess_text)


# Function to extract embeddings from BERT
def extract_embeddings(text, model, tokenizer, max_length=512):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()  # Mean pooling over token embeddings
    return embedding


# Function to save embeddings to a pickle file
def save_embeddings(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f"Saved embeddings to {filename}")


# Function to generate and save embeddings for each column
def generate_and_save_embeddings(df, column_name):
    print(f"Processing column: {column_name}")
    column_data = df[column_name].dropna()  # Remove any NaN values if present

    # BERT embeddings
    embeddings = np.array([extract_embeddings(text, bert_model, bert_tokenizer) for text in column_data])
    save_embeddings(embeddings, f'/kaggle/working/{column_name}_bert.pkl')
    print(f"SAVED IS  {column_name}")
    return embeddings


# Generate and save embeddings for each column
prompt_embeddings = generate_and_save_embeddings(train, 'prompt')
response_a_embeddings = generate_and_save_embeddings(train, 'response_a')
response_b_embeddings = generate_and_save_embeddings(train, 'response_b')



# Define function to calculate metrics
def calculate_metrics(prompt_emb, response_a_emb, response_b_emb):
    metrics = {}

    # 1. Cosine Similarity
    metrics['cosine_similarity_a'] = cosine_similarity([prompt_emb], [response_a_emb])[0][0]
    metrics['cosine_similarity_b'] = cosine_similarity([prompt_emb], [response_b_emb])[0][0]

    # 2. Cosine Distance
    metrics['cosine_distance_a'] = 1 - metrics['cosine_similarity_a']
    metrics['cosine_distance_b'] = 1 - metrics['cosine_similarity_b']

    # 3. Mean Euclidean Distance
    metrics['euclidean_distance_a'] = euclidean_distances([prompt_emb], [response_a_emb])[0][0]
    metrics['euclidean_distance_b'] = euclidean_distances([prompt_emb], [response_b_emb])[0][0]

    # 4. Semantic Overlap (measured by cosine similarity)
    metrics['semantic_overlap_a'] = metrics['cosine_similarity_a']
    metrics['semantic_overlap_b'] = metrics['cosine_similarity_b']

    # 5. Topic Coherence (same as semantic overlap for now)
    metrics['topic_coherence_a'] = metrics['cosine_similarity_a']
    metrics['topic_coherence_b'] = metrics['cosine_similarity_b']

    # 6. Prompt Adherence
    metrics['prompt_adherence_a'] = metrics['cosine_similarity_a']
    metrics['prompt_adherence_b'] = metrics['cosine_similarity_b']

    # 7. Contextual Consistency
    metrics['contextual_consistency'] = cosine_similarity([response_a_emb], [response_b_emb])[0][0]

    # 8. Diversity of Response
    metrics['response_diversity'] = 1 - metrics['contextual_consistency']

    # 9. Sentiment Similarity Score
    metrics['sentiment_similarity_a'] = metrics['cosine_similarity_a']  # Proxy
    metrics['sentiment_similarity_b'] = metrics['cosine_similarity_b']  # Proxy

    # 10. Topic Coverage (same as semantic overlap for simplicity)
    metrics['topic_coverage_a'] = metrics['cosine_similarity_a']
    metrics['topic_coverage_b'] = metrics['cosine_similarity_b']


    # 12. Depth of Response (Variance of embeddings as proxy)
    metrics['depth_of_response_a'] = np.var(response_a_emb)
    metrics['depth_of_response_b'] = np.var(response_b_emb)

    # 13. Focus Score (Inverse of Euclidean Distance)
    metrics['focus_score_a'] = 1 / (1 + metrics['euclidean_distance_a'])
    metrics['focus_score_b'] = 1 / (1 + metrics['euclidean_distance_b'])

    # 14. Embedding Vector Magnitude Ratio
    metrics['embedding_magnitude_ratio_a'] = np.linalg.norm(response_a_emb) / np.linalg.norm(prompt_emb)
    metrics['embedding_magnitude_ratio_b'] = np.linalg.norm(response_b_emb) / np.linalg.norm(prompt_emb)

    # 15. Off-Topic Score
    metrics['off_topic_a'] = metrics['cosine_distance_a']
    metrics['off_topic_b'] = metrics['cosine_distance_b']


    # 16. Euclidean Distance
    metrics['euclidean_distance_a'] = euclidean_distances([prompt_emb], [response_a_emb])[0][0]
    metrics['euclidean_distance_b'] = euclidean_distances([prompt_emb], [response_b_emb])[0][0]

    # 17. Embedding Density
    metrics['embedding_density_a'] = np.sum(np.square(response_a_emb))
    metrics['embedding_density_b'] = np.sum(np.square(response_b_emb))

    # 18. Internal Coherence
    metrics['internal_coherence_a'] = np.mean(cosine_similarity([response_a_emb]))
    metrics['internal_coherence_b'] = np.mean(cosine_similarity([response_b_emb]))

    # 19. Response Concreteness
    metrics['response_concreteness_a'] = np.std(response_a_emb)
    metrics['response_concreteness_b'] = np.std(response_b_emb)

    # 20. Redundancy Score
    metrics['redundancy_score_a'] = 1 - np.mean(np.std(response_a_emb))
    metrics['redundancy_score_b'] = 1 - np.mean(np.std(response_b_emb))

    # 21. Semantic Entailment (similar to cosine similarity)
    metrics['semantic_entailment_a'] = metrics['cosine_similarity_a']
    metrics['semantic_entailment_b'] = metrics['cosine_similarity_b']

    # 22. Prompt-Response Embedding Ratio
    metrics['embedding_ratio_a'] = np.linalg.norm(response_a_emb) / np.linalg.norm(prompt_emb)
    metrics['embedding_ratio_b'] = np.linalg.norm(response_b_emb) / np.linalg.norm(prompt_emb)

    # 23. Informativeness Score
    metrics['informativeness_a'] = np.var(response_a_emb)
    metrics['informativeness_b'] = np.var(response_b_emb)

    # 24. Topic Drift Score
    metrics['topic_drift_a'] = np.std(response_a_emb - prompt_emb)
    metrics['topic_drift_b'] = np.std(response_b_emb - prompt_emb)

    # 25. Context Alignment
    metrics['context_alignment'] = cosine_similarity([response_a_emb], [response_b_emb])[0][0]

    #26. Disparity Score
    metrics['disparity_score'] = euclidean_distances([response_a_emb], [response_b_emb])[0][0]


    return metrics


def load_data(prompt_path,response_a_path , response_b_path):
    # Load precomputed embeddings (assuming they've been generated and saved as instructed)
    prompt_embeddings = pickle.load(open(prompt_path, 'rb'))
    response_a_embeddings = pickle.load(open(response_a_path, 'rb'))
    response_b_embeddings = pickle.load(open(response_b_path , 'rb'))

    train_features_data = []
    for i in range(len(prompt_embeddings)):
        metrics = calculate_metrics(prompt_embeddings[i], response_a_embeddings[i], response_b_embeddings[i])
        train_features_data.append(metrics)
    train_features_data = pd.DataFrame(train_features_data)
    return train_features_data

prompt_path = '/kaggle/input/training-llm-prompt-classifciation-bert-embedding/prompt_bert.pkl'
response_a_path = '/kaggle/input/training-llm-prompt-classifciation-bert-embedding/response_a_bert.pkl'
response_b_path  =  '/kaggle/input/training-llm-prompt-classifciation-bert-embedding/response_b_bert.pkl'
train_features_from_embedding = load_data(prompt_path,response_a_path , response_b_path)

prompt_path = '/kaggle/working/prompt_bert.pkl'
response_a_path ='/kaggle/working/response_a_bert.pkl'
response_b_path = '/kaggle/working/response_b_bert.pkl'
test_features_from_embedding = load_data(prompt_path,response_a_path , response_b_path)

print('======================== TRAIN_FEATURES AND TEST_FEATURES LOADED ===============================')



In [ ]:
print('===================Training and submission ceation =======')

# Import necessary libraries
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
import os

# Load training data
train_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
test_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")

# Encode the target columns
train_df['target'] = np.select(
    [
        train_df['winner_model_a'] == 1,
        train_df['winner_model_b'] == 1,
        train_df['winner_tie'] == 1
    ],
    [0, 1, 2],
    default=-1  # Assign -1 for any invalid cases
)

# Verify that there are no invalid targets
if (train_df['target'] == -1).any():
    raise ValueError("There are invalid target labels in the training data.")

# Define target variable
y = train_df['target']


# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    train_features_from_embedding,
    y,
    test_size=0.02,
    random_state=42,
    stratify=y
)


# Initialize GradientBoostingClassifier
gb_model = GradientBoostingClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42
)

# Train the GradientBoosting model
print("Training GradientBoosting model...")
gb_model.fit(X_train, y_train)
print("GradientBoosting training completed.")

# Validate the GradientBoosting model
print("Evaluating GradientBoosting model on validation set...")
gb_val_predictions = gb_model.predict(X_val)
print("Classification Report for GradientBoosting:")
print(classification_report(y_val, gb_val_predictions))

# Make predictions on the test set using GradientBoosting
print("Making predictions on test data with GradientBoosting...")
gb_test_predictions = gb_model.predict_proba(test_features_from_embedding)

# Ensure that gb_test_predictions has the correct number of classes
num_classes = len(gb_model.classes_)
if gb_test_predictions.shape[1] != num_classes:
    raise ValueError(f"Expected {num_classes} classes, but got {gb_test_predictions.shape[1]}.")

# Create GradientBoosting submission DataFrame
gb_submission = pd.DataFrame(gb_test_predictions, columns=['winner_model_a', 'winner_model_b', 'winner_tie'])
gb_submission['id'] = test_df['id']

# Reorder columns for submission
gb_submission = gb_submission[['id', 'winner_model_a', 'winner_model_b', 'winner_tie']]

# Save the GradientBoosting submission file
gb_submission_file = '/kaggle/working/submission.csv'
gb_submission.to_csv(gb_submission_file, index=False)
print(f"GradientBoosting submission saved to {gb_submission_file}")

# Optionally, display the first few rows of the GradientBoosting submission file
print("First few rows of the GradientBoosting submission file:")
print(gb_submission.head())


In [ ]:
# import spacy
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# from nltk.translate.bleu_score import sentence_bleu
# from sklearn.metrics.pairwise import cosine_similarity
# from nltk.translate.bleu_score import sentence_bleu
# from collections import Counter


# # Helper functions for different metrics
# def jaccard_similarity(text1, text2):
#     words1 = set(text1.split())
#     words2 = set(text2.split())
#     return len(words1 & words2) / len(words1 | words2)

# def lexical_diversity(text):
#     words = text.split()
#     return len(set(words)) / len(words)

# def topic_modeling_score(text1, text2):
#     # This can be replaced with actual topic modeling logic (e.g., using LDA or BERT embeddings)
#     return cosine_similarity(model.encode([text1]), model.encode([text2]))[0][0]


# def lexical_variety(text):
#     words = text.split()
#     return len(set(words)) / len(words)


# # Initialize Spacy NLP model
# nlp = spacy.load('en_core_web_sm')


# # Function to compute cosine similarity (Relevance)
# def calculate_relevance(prompt, response):
#     vectorizer = TfidfVectorizer(stop_words='english')
#     tfidf_matrix = vectorizer.fit_transform([prompt, response])
#     similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
#     return similarity[0][0] * 10  # Scale to 0-10 range

# # Function to calculate coherence (using sentence length and structure)
# def calculate_coherence(response):
#     doc = nlp(response)
#     sentence_lengths = [len(sent) for sent in doc.sents]
#     avg_sentence_length = sum(sentence_lengths) / len(sentence_lengths) if sentence_lengths else 0
#     return min(10, avg_sentence_length / 10)



# # Function to calculate creativity (based on vocabulary diversity)
# def calculate_creativity(response):
#     doc = nlp(response)
#     unique_words = set([token.text.lower() for token in doc if token.is_alpha])
#     return min(10, len(unique_words) / 5)

# # Function to calculate engagement (based on the use of questions)
# def calculate_engagement(response):
#     doc = nlp(response)
#     questions = [sent for sent in doc.sents if '?' in sent.text]
#     return min(10, len(questions))

# # Function to calculate conciseness (based on word count)
# def calculate_conciseness(response):
#     word_count = len(response.split())
#     return max(0, 10 - word_count / 10)

# # Function to calculate fluency and naturalness (based on average sentence length and grammar)
# def calculate_fluency(response):
#     doc = nlp(response)
#     sentence_lengths = [len(sent) for sent in doc.sents]
#     avg_sentence_length = sum(sentence_lengths) / len(sentence_lengths) if sentence_lengths else 0
#     return min(10, avg_sentence_length / 10)

# # Function to calculate completeness (based on the presence of main points)
# def calculate_completeness(prompt, response):
#     # Check if the response covers key points mentioned in the prompt
#     response_tokens = set(response.lower().split())
#     prompt_tokens = set(prompt.lower().split())
#     common_tokens = response_tokens.intersection(prompt_tokens)
#     completeness_score = len(common_tokens) / len(prompt_tokens) * 10
#     return min(10, completeness_score)

# # BLEU Score function
# def calculate_bleu(reference, response):
#     reference = [reference.split()]
#     response = response.split()
#     return sentence_bleu(reference, response) * 10  # Scale to 0-10 range


# # Levenshtein (Edit Distance) Score function
# def calculate_levenshtein(reference, response):
#     return (1 - Levenshtein.distance(reference, response) / max(len(reference), len(response))) * 10  # Scale to 0-10 range



# def evaluate_response(prompt, response_1, response_2):
#     # Calculate all metrics for Response 1
#     metrics_response = {
#         'response_a_Coherence': calculate_coherence(response_1),
#         'response_a_Completeness': calculate_completeness(prompt, response_1),
#         'response_a_Engagement': calculate_engagement(response_1),
#         'response_a_Conciseness': calculate_conciseness(response_1),
#         'response_a_Creativity': calculate_creativity(response_1),
#         'response_a_Fluency and Naturalness': calculate_fluency(response_1),
#         'response_a_BLEU': calculate_bleu(prompt, response_1),
#         'response_a_jaccard_1' : jaccard_similarity(prompt, response_1),
#         'response_a_length_1' : len(response_1.split()),
#         'response_a_lexical_1' : lexical_diversity(response_1),
#         'response_b_Coherence': calculate_coherence(response_2),
#         'response_b_Completeness': calculate_completeness(prompt, response_2),
#         'response_b_Engagement': calculate_engagement(response_2),
#         'response_b_Conciseness': calculate_conciseness(response_2),
#         'response_b_Creativity': calculate_creativity(response_2),
#         'response_b_Fluency and Naturalness': calculate_fluency(response_2),
#         'response_b_BLEU': calculate_bleu(prompt, response_2),
#         'response_b_jaccard' : jaccard_similarity(prompt, response_2),
#         'response_b_length' :len(response_2.split()),
#         'response_b_lexical' : lexical_diversity(response_2),
#     }
    
#     return metrics_response


# def load_text_features(file_path):
#     df = pd.read_csv(file_path, usecols=['prompt', 'response_a', 'response_b'])
#     features_list = []
#     for index, row in df.iterrows():
#         prompt = row['prompt']
#         response_a = row['response_a']
#         response_b = row['response_b']
        
#         features_row_dict = evaluate_response(prompt, response_a, response_b)
#         features_list.append(features_row_dict)
    
#     features_df = pd.DataFrame(features_list)
#     return  features_df
    

# import pandas as pd
# train_path = '/kaggle/input/llm-classification-finetuning/train.csv'
# train_text_features = load_text_features(train_path)
# test_path = '/kaggle/input/llm-classification-finetuning/test.csv'
# test_text_features = load_text_features(test_path)
# print(train_text_features.head(10))